In [1]:
import helper_fxns as hf
import importlib
import pandas as pd
import numpy as np
import os
import requests
import sys
%matplotlib inline

In [ ]:
#img, scalings = make_isometric(img, max_dim_len=)

#(x,y,z) = scale_xyz(x,y,z, scalings)

In [3]:
def flip_z(z, img):
    return (img.shape[2]-z[1], img.shape[2]-z[0])

def flip_y(y, img):
    return (img.shape[1]-y[1], img.shape[1]-y[0])

#def make_isometric(img, dims):
#def scale_xyz(x,y,z, scalings):  

In [5]:
rois = []
def add_roi(mrn, x, y, z, cls):
    rois.append([str(mrn) + ".npy", x[0], y[0], z[0], x[1], y[1], z[1], cls])

In [4]:
def add_data(mrn,x,y,z,cls):
    img = np.transpose(np.stack((pre[mrn][0], art[mrn][0])), (1,2,3,0))
    img_dir = "imgs"
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
    np.save(img_dir + "\\" + mrn, img)

    y = flip_y(y,img)
    z = flip_z(z,img)
    add_roi(mrn, x,y,z, cls)

In [76]:
def validate_dims(imgs, mrns):
    """Validate that all series of the same MRN have the same dimensions.
    Images formatted as list of dicts. Each dict is a different image channel, indexed by MRN."""
    for mrn in mrns:
        for x in range(1,len(imgs)):
            assert imgs[0][mrn][0].shape == imgs[x][mrn][0].shape \
                   and np.all(imgs[0][mrn][1] == imgs[x][mrn][1]), "img %s has wrong shape" % mrn

In [83]:
importlib.reload(hf)

<module 'helper_fxns' from 'C:\\Users\\Clinton Wang\\Documents\\liver-mr-processor\\helper_fxns.py'>

In [37]:
def preprocess_df(df):
    df.drop(set(df.columns).difference(['Patient E Number', 
          'x1', 'x2', 'y1', 'y2', 'z1', 'z2', 'Image type']), axis=1, inplace=True)
    return df[df['Run'] == 1].dropna(subset=["x1"])

In [54]:
subdir = base_dir+"\\12229245"

In [102]:
def load_imgs(df, base_dir, cls):
    df = preprocess_df(df)
    mrns = list(df['Patient E Number'].astype(str))
    for mrn in mrns:
        subdir = base_dir+"\\"+mrn
        try:
            pre[mrn] = hf.dcm_load(subdir+r"\T1_BL")
            art[mrn] = hf.dcm_load(subdir+r"\T1_AP")
            #ven[mrn] = hf.dcm_load(subdir+r"\T1_VP")
            #equ[mrn] = hf.dcm_load(subdir+r"\T1_EQ")
            #t2[mrn] = hf.dcm_load(subdir+r"\T2")
        except Exception as e:
            print("accession number %s encountered error" % mrn, str(e))
            continue
            
    validate_dims([pre, art], mrns)
    
    for _, row in df.iterrows():
        add_data(str(row['Patient E Number']), x=(row['x1'], row['x2']),
                 y=(row['y1'], row['y2']), z=(row['z1'], row['z2']), cls)

SyntaxError: positional argument follows keyword argument (<ipython-input-102-04b05dfb9d96>, line 20)

In [84]:
pre = {}
art = {}
#ven = {}
#equ = {}
#t2 = {}

base_dir = r'raw_imgs\simple_cysts'
df = pd.read_excel(r'raw_imgs\Prototype1.xlsx', sheetname="Hepatic Cyst")
load_imgs(df, base_dir=r'raw_imgs\simple_cysts', cls='cyst')

In [90]:
base_dir = r'raw_imgs\OPTN5A'
df = pd.read_excel(r'raw_imgs\Prototype1.xlsx', sheetname="OPTN 5A")
load_imgs(df, base_dir = r'raw_imgs\OPTN5A', cls='hcc')

In [109]:
with open('train_list.txt', 'w') as f:
    for roi in rois:
        f.write(",".join([str(x) for x in roi]) + "\n")

In [ ]:
def draw_fig(image_array, name):
    """Draw an image of type np array and save it to."""

    image_array = image_array[:,:,slice_index]
    aspect = float(image_array.shape[1]) / image_array.shape[0]
    w = 20
    h = int(aspect * w)

    image_array = np.rot90(image_array)

    fig = plt.figure(frameon=False)
    fig.set_size_inches(w,h)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(image_array, interpolation='bilinear', norm=cnorm, cmap=plt.cm.gray, aspect='auto')

    img_fname = name + '.png'
    plt.savefig(img_fname)
    print('Segmentation saved as %s' % img_fname)

In [ ]:
def create_features(name, precontrast, arterial, liver_mask_file, tumor_mask_file, format_output, header_output, seg, slice_index):
    diff = art_data - pre_data # Calculate the subtracted image.
    diff[diff < 0] = 0.0  # The pre-contrast should never be more than the arterial. Clamp negative values to zero.

    x, y, z = diff.shape

    ##### Begin code for drawing specific mask slices. ######
    cnorm = matplotlib.colors.Normalize(vmin=0, vmax=np.amax(diff))

    draw_fig(diff, 'whole')

    # Calculate the liver volume.
    liver_volume = np.sum(liver) * pre_dimx * pre_dimy * pre_dimz
    is_in_mm = pre_units == 2 or liver_volume > 10000
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        liver_volume /= 1000

    tumor_volume = np.sum(tumor) * pre_dimx * pre_dimy * pre_dimz
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        tumor_volume /= 1000

    mean_intensity = np.mean(diff[img > 0])
    std_intensity = np.std(diff[img > 0])

    # Calculate the volume of tumor that whose voxel intensities are greater than the mean liver intensity.
    # Note this is NOT the same threshold as qEASL (which would need a parenchymal ROI in order to define the intensity threshold).
    enhancing_tumor_volume = np.sum(tumor[just_tumor > mean_liver_intensity]) * pre_dimx * pre_dimy * pre_dimz
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        enhancing_tumor_volume /= 1000

    headers = ['Name', 'Liver volume', 'Mean liver intensity', 'STD liver intensity', 'Tumor volume', 'Mean tumor intensity', 'STD tumor intensity', 'Enhancing tumor volume']
    features = [name, liver_volume, mean_liver_intensity, std_liver_intensity, tumor_volume, mean_tumor_intensity, std_tumor_intensity, enhancing_tumor_volume]
    if format_output == 'tabulate':
        if header_output:
            results = tabulate.tabulate([features], headers=headers)
        else:
            results = tabulate.tabulate([features])
    else:
        features_row = '\t'.join(str(feature) for feature in features)
        if header_output:
            results = '\t'.join(headers) + '\n' + features_row
        else:
            results = features_row

    print(results)

In [ ]:
def training_pipeline(pre_dcm_fns, post_dcm_fns, labels):
    """
    pre_dcm_fn is the precontrast image, and post_dcm_fn is the postcontrast image
    1. Import pre/20s DICOMs
    2. Register pre to 20s
    3. Segment whole liver
    4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
    5. For HCC labels, calculate and store embeddings (feature values)
    6. Train CNN on all
    7. Train feature engineered network on HCC labels
    """
    train_frac = 0.7
    data_size = len(pre_dcm_fns)
    
    # TODO: do a proper k-fold x-validation split
    X_cnn_train = X_cnn[:data_size*train_frac]
    X_cnn_val = X_cnn[data_size*train_frac:]
    
    for x in range(data_size):
        pre_dcm_fn = pre_dcm_fns[x]
        post_dcm_fn = post_dcm_fn[x]
        label = labels[x]
        
        pre_img = dcm_load(pre_dcm_fn)
        post_img = dcm_load(post_dcm_fn)

        pre_img = seg_liver(pre_img)
        post_img = seg_liver(post_img)
        
        y_cnn = label['is-hcc']
        label.pop('is-hcc')
        X_cnn = [pre_img, post_img, label]
        
        """if y_cnn:
            features = get_features(pre_img, post_img, label)
            
            # rfm is random forest model
            y_rfm = label
            X_rfm = [pre_img, post_img, label]"""

# Overall approach
This is a tool for classifying precontrast and 20s postcontrast MRIs (into LI-RADS stages / predicting survival?).

#### Training
-In mini-batches:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
5. For HCC labels, calculate and store embeddings (feature values)
6. Train CNN on all

-After CNN is trained, then:
7. Train feature engineered network on HCC labels

#### Test pipeline:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Use CNN to classify as HCC/non-HCC
5. If HCC, calculate features values
6. If HCC, use feature engineered network to predict label

#### CNN models
First, try VGGNet, then ResNet. Then try VAE-GAN, training on benign only; deviation score = malignancy.